In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [27]:
file_path = '/content/drive/MyDrive/hitters_processed.pkl'
data = pd.read_pickle(file_path)

In [28]:
# Hedef değişken ve bağımsız değişkenleri ayır

X = data.drop(columns=['Salary'])
y = data['Salary']

In [29]:
# train,test böl

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
linear_model= LinearRegression()
ridge_model = Ridge(alpha=1.0)  # Ridge parametresi alpha
lasso_model = Lasso(alpha=0.1)  # Lasso parametresi alpha

# Modelleri eğit
linear_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)
lasso_model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
linear_preds = linear_model.predict(X_test)
ridge_preds = ridge_model.predict(X_test)
lasso_preds = lasso_model.predict(X_test)

# Modellerin performansını değerlendir
results = pd.DataFrame({
    "Model": ["Linear Regression", "Ridge Regression", "Lasso Regression"],
    "MSE": [
        mean_squared_error(y_test, linear_preds),
        mean_squared_error(y_test, ridge_preds),
        mean_squared_error(y_test, lasso_preds)
    ],
    "R2 Score": [
        r2_score(y_test, linear_preds),
        r2_score(y_test, ridge_preds),
        r2_score(y_test, lasso_preds)
    ]
})

# Sonuçları yazdır
print(results)


               Model           MSE  R2 Score
0  Linear Regression  42335.518494  0.698201
1   Ridge Regression  48202.134026  0.656380
2   Lasso Regression  41841.474791  0.701723


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.006e+06, tolerance: 3.462e+03
  model = cd_fast.enet_coordinate_descent(


**Linear Regresyon**
* MSE, tahminlerle gerçek değerler arasındaki farkların karelerinin ortalaması. Yani ne kadar düşükse o kadar iyi. R² ise modelin bağımsız değişkenlerin bağımlı değişkeni açıklama oranını gösteriyor, 0 ile 1 arasında ve yüksek olması iyi.R^2'nin yaklaşık %70 olması modelin iyi çalıştığını gösteriyor ama MSE'nin yüksek olması tahminlerde önemli hatalar olabileceğine işaret ediyor.

**Ridge Regresyon**
* Linear ve Lassoya göre daha yüksek bir MSE'ye sahip,bu Ridge'in tahminlerinde daha belirsiz olduğunu gösteriyor.

**Lasso Regresyon**
* Ridge'e göre daha kesin tahminler yapıyor. Linear'le karşılaştırıldığında hem daha düşük MSE hem de daha yüksek bir R^2 olduğu için daha iyi performans gösteriyor.


In [31]:
#En iyi alpha değerlerini bulup modeli tekrardan eğitiyoruz

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd


# Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)

# Define the alpha values to test
alpha_values = {'ridge__alpha': [0.01, 0.1, 1.0, 10.0, 100.0], 'lasso__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]}

# Create Ridge and Lasso pipelines
ridge_pipeline = Pipeline([
    ('poly', poly),
    ('ridge', Ridge())
])

lasso_pipeline = Pipeline([
    ('poly', poly),
    ('lasso', Lasso(max_iter=10000))
])

# Perform GridSearchCV for Ridge
ridge_search = GridSearchCV(ridge_pipeline, param_grid={'ridge__alpha': alpha_values['ridge__alpha']}, cv=5, scoring='neg_mean_squared_error')
ridge_search.fit(X_train, y_train)

# Perform GridSearchCV for Lasso
lasso_search = GridSearchCV(lasso_pipeline, param_grid={'lasso__alpha': alpha_values['lasso__alpha']}, cv=5, scoring='neg_mean_squared_error')
lasso_search.fit(X_train, y_train)

# Best alpha values
best_ridge_alpha = ridge_search.best_params_['ridge__alpha']
best_lasso_alpha = lasso_search.best_params_['lasso__alpha']

# Refit models with the best alpha values
ridge_best_model = ridge_search.best_estimator_
lasso_best_model = lasso_search.best_estimator_

# Predictions
ridge_best_preds = ridge_best_model.predict(X_test)
lasso_best_preds = lasso_best_model.predict(X_test)

# Evaluate models
results = pd.DataFrame({
    "Model": ["Optimized Ridge Regression", "Optimized Lasso Regression"],
    "Best Alpha": [best_ridge_alpha, best_lasso_alpha],
    "MSE": [
        mean_squared_error(y_test, ridge_best_preds),
        mean_squared_error(y_test, lasso_best_preds)
    ],
    "R2 Score": [
        r2_score(y_test, ridge_best_preds),
        r2_score(y_test, lasso_best_preds)
    ]
})

print(results)



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.017e+03, tolerance: 2.637e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.798e+03, tolerance: 2.755e+03
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.080e+04, tolerance: 2.671e

                        Model  Best Alpha           MSE  R2 Score
0  Optimized Ridge Regression       100.0  65734.922802  0.531393
1  Optimized Lasso Regression        10.0  64507.414295  0.540144


Her iki modelin de optimize edilmiş olmasına rağmen performansları önceki standart Ridge/Lasso modellerine göre düştü. MSE'nin 64k-65k bandına çıkması ve R^2'nin 0.53-0.54 seviyesinde kalması, modellerin veriyi yeterince öğrenemediğini gösteriyor.